In [1]:
import pandas as pd
import torch
from transformers import  T5ForConditionalGeneration,Adafactor, T5Tokenizer,AutoTokenizer
import time
import warnings
warnings.filterwarnings('ignore')
from IPython.display import HTML, display
import os

In [12]:
model_checkpoint="../train_t5/flanT5checkpoint100.bin"

In [13]:
# model_phoneme_checkpoint = "../checkpoint/full_phoneme_split_normal_weight_decay_lr_4_T5_small/flanT5checkpoint400.bin"

In [14]:
model=T5ForConditionalGeneration.from_pretrained(model_checkpoint,return_dict=True,config="../config_flanT5_small.json")

In [109]:
# model_phoneme = T5ForConditionalGeneration.from_pretrained(model_phoneme_checkpoint,return_dict=True,config="./config_flanT5_small.json")

In [10]:
model_config="google/flan-t5-small"

In [11]:
tokenizer=AutoTokenizer.from_pretrained(model_config)

In [119]:
val_df=pd.read_csv('../test.csv')
# val_df = pd.read_csv('../validation.csv')

In [120]:
val_df

,Unnamed: 0,city,term,country
0,0,pyongyang,pyongyangite,North Korea
1,1,putney,putneyite,NaN
2,2,winston salem,winston salemite,United States
3,3,chennai,chennaiite,India
4,4,ann arbor,ann arborite,NaN
...,...,...,...,...
157,157,rampur,rampuri,India
158,158,bijaipur,bijapuri,India
159,159,birganj,birganji,Nepal
160,160,kaposvar,kaposvari,Hungary


In [121]:
def remove_sep(x):
   x = x.replace('<SEP>','')
  #  print(x)
   return x
def normal(df):
   df["city"]= df['city'].map(remove_sep)

In [122]:
normal(val_df)

In [68]:
# from g2p_en import G2p

# phoneme_list = []

# def word_to_ipa(word):
#     g2p = G2p()  # Initialize the G2P model
#     ipa = g2p(word)  # Convert the word to IPA
#     return " ".join(ipa)

# ipa = word_to_ipa("tokyo")
# phoneme_list.append(ipa)
# print(ipa)

In [123]:
def generate(word):
    input_ids=tokenizer.encode('predict demonym: ',word,return_tensors="pt")
    outputs=model.generate(input_ids)
    gen_text=tokenizer.decode(outputs[0]).replace('<pad>','').replace('</s>','')
    gen_text=gen_text.strip()
    return gen_text

In [70]:
# def generate_phoneme(word,phoneme):
#     input_ids=tokenizer.encode('predict demonym: ',word + '<SEP>' + phoneme,return_tensors="pt")
#     outputs=model_phoneme.generate(input_ids)
#     gen_text=tokenizer.decode(outputs[0]).replace('<pad>','').replace('</s>','')
#     gen_text=gen_text.strip()
#     return gen_text


In [71]:
# generate("newyork")

'newyorker'

In [72]:
# !pip install gradio

In [124]:
from tqdm import tqdm

correctcount = 0
totalcount = len(val_df)
prediction = []
yesno = []

for _, row in tqdm(val_df.iterrows(), total=totalcount, desc="Processing"):
    # print(row[1])
    ans = generate(row[1])
    prediction.append(ans)
    special_character = "\u2581"  # Unicode code point for the character ▁
    val = row[2].replace(special_character, "").strip()
    if ans == val:
        correctcount += 1
        yesno.append('yes')
    else:
        yesno.append('no')

accuracy = (correctcount / totalcount) * 100
print("accuracy:", accuracy)

Processing: 100%|██████████| 162/162 [00:12<00:00, 12.71it/s]

accuracy: 42.592592592592595


In [125]:
import gradio as gr  # Replace with your model import

In [27]:
def predict(text):
    prediction=generate(text)
    return prediction

In [126]:
outputs = gr.outputs.Textbox()
app = gr.Interface(fn=predict, inputs=['text'], outputs=outputs,description="Inhabitant Prediction Model")

In [ ]:
app.launch()